In [1]:
# librerias
import pdfplumber
import pandas as pd
import re
import json
from rapidfuzz import process, fuzz

In [2]:
# importar datos diccionario
df_maestro = pd.read_csv('../data/diccionario_maestro.csv', sep=',', encoding='utf-8')
df_maestro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      70 non-null     int64 
 1   nombre  70 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [3]:
# explorar headers
def inspeccionar_estructura_pdf(path):
    with pdfplumber.open(path) as pdf:
        pagina = pdf.pages[0]
        texto_completo = pagina.extract_text()
        tabla = pagina.extract_table()
        
        titulos_tabla = []
        if tabla:
            titulos_tabla = [str(c).replace('\n', ' ') for c in tabla[0] if c is not None]
        
        patente = re.search(r'[A-Z]{2}\s?\d{3}\s?[A-Z]{2}|[A-Z]{3}\s?\d{3}', texto_completo)
        fecha = re.search(r'\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4}', texto_completo)
        
        analisis = {
            "columnas_detectadas": titulos_tabla,
            "patente_encontrada": patente.group(0) if patente else "No detectada",
            "fecha_encontrada": fecha.group(0) if fecha else "No detectada",
            "preview_texto": texto_completo[:500]
        }
        
        return analisis

path_test = "../data/presupuesto_A.pdf"
info_pdf = inspeccionar_estructura_pdf(path_test)

print("--- ANÁLISIS DE ESTRUCTURA ---")
print(f"Columnas halladas: {info_pdf['columnas_detectadas']}")
print(f"Patente: {info_pdf['patente_encontrada']}")
print(f"Fecha: {info_pdf['fecha_encontrada']}")

--- ANÁLISIS DE ESTRUCTURA ---
Columnas halladas: ['TIPO', 'DESCRIPCION', 'CANT.', 'DSCTO', 'PRECIO', 'TOTAL']
Patente: DOR 184
Fecha: 14-01-2026


In [4]:
def identificar_taller_y_metadatos(path):
    with pdfplumber.open(path) as pdf:
        texto_completo = pdf.pages[0].extract_text()
        tabla = pdf.pages[0].extract_table()
        
        # 1. Identificar Taller por columnas únicas
        columnas = [str(c).replace('\n', ' ') for c in tabla[0] if c is not None]
        taller = "DESCONOCIDO"
        if "Código" in columnas: taller = "WAGEN"
        elif "TIPO" in columnas: taller = "TREOS"
        
        # 2. Regex Mejorado para Patentes (Evita capturar códigos de piezas)
        # Busca 3 letras + espacio/guion + 3 números O 2 letras + 3 números + 2 letras
        patente_regex = r'\b([A-Z]{3}\s?\d{3})\b|\b([A-Z]{2}\s?\d{3}\s?[A-Z]{2})\b'
        matches = re.findall(patente_regex, texto_completo)
        # Limpiar el resultado del findall (tuplas)
        patente_final = "No detectada"
        if matches:
            # Tomamos el primer match que no sea nulo
            patente_final = [item for sublist in matches for item in sublist if item][0]

        return {
            "taller": taller,
            "columnas": columnas,
            "patente": patente_final,
            "texto_base": texto_completo
        }

# Probar con ambos
info_wagen = identificar_taller_y_metadatos("../data/presupuesto_B.pdf")
info_treos = identificar_taller_y_metadatos("../data/presupuesto_A.pdf")

print(f"WAGEN -> Patente: {info_wagen['patente']} | Taller detectado: {info_wagen['taller']}")
print(f"TREOS -> Patente: {info_treos['patente']} | Taller detectado: {info_treos['taller']}")

WAGEN -> Patente: AB082MN | Taller detectado: WAGEN
TREOS -> Patente: No detectada | Taller detectado: TREOS


In [5]:
import pdfplumber

def extraer_texto_completo(path_pdf):
    texto_total = ""
    
    with pdfplumber.open(path_pdf) as pdf:
        # Iteramos por cada página del documento
        for i, pagina in enumerate(pdf.pages):
            texto_pagina = pagina.extract_text()
            
            if texto_pagina:
                texto_total += f"--- PÁGINA {i+1} ---\n"
                texto_total += texto_pagina + "\n\n"
    
    return texto_total

# Uso en tu notebook
path = "../data/presupuesto_C.pdf" # Cambia por el nombre de tu archivo
texto_presupuesto = extraer_texto_completo(path)

# Imprimir los primeros 1000 caracteres para verificar
print(texto_presupuesto[:10000])

--- PÁGINA 1 ---
CARROCERIA Y PINTURA
Documento no Valido como factura
"EQUIPO C"
PRESUPUESTO Nº 5028
Echeandia 3084- C1406HRJ C.A.B.A Fecha de emisión 1 de julio de 2025
Tel.4613-0720 - tallerequipoc@gmail.com
RESPONSABLE INSCRIPTO C.U.I.T Nº 20-35316807-0 - Ing.Brutos Nº1619348-2 - Inic.Activ: 01/05/2022
Señor: Cia. Seguro:
Calle: Autom.Marca: CHEVROLET CRUZE
Localidad: Dominio: AD 184 VR
C: CAMBIAR - R: REPARAR - P: PINTAR
PARTE DELANTERA PRECIO C R P PARTE TRASERA PRECIO C R P
CHASIS DELANTERO: DER CHASIS TRASERO: DER - IZQ
ALMA DE PARAGOLPE ALMA DE PARAGOLPE x
PARAGOLPE-CENTRO-PUNTERA-BANDA PARAGOLPE CENTRO-PUNTERA-BANDA X
SOPORTE DE PARAGOLPE PANEL DE COLA X
REJILLA DE FRENTE PORTON - TAPA DE BAUL X
GUARDAPLAST: DER - IZQ PISO BAUL - INTERIOR
FRENTE: SUP - INF GUARDABARRO: DER - IZQ
TRAVESAÑO PASARRUEDA: DER - IZQ
CAPOT GUARDAPLAST: DER - IZQ
PARRILLA DE FRENTE BAJADA TECHO: DER - IZQ
GURDABARRO: DER - IZQ LUNETA - BURLETE
PASARRUEDA: DER - IZQ MOLDURA PUERTA: DER - IZQ
PARANTE: 

In [6]:
def extraer_texto_completo(path_pdf, incluir_layout=False):
    """
    Extrae todo el texto de un PDF página por página.
    Opcionalmente usa layout=True para preservar mejor el orden en tablas y columnas.
    """
    texto_total = []
    
    try:
        with pdfplumber.open(path_pdf) as pdf:
            print(f"Total de páginas: {len(pdf.pages)}")
            
            for i, pagina in enumerate(pdf.pages, 1):
                # Opción 1: extraer texto simple
                texto = pagina.extract_text(layout=incluir_layout)
                
                # Opción 2: si extract_text falla o da muy poco → probar extract_text_lines
                if not texto or len(texto.strip()) < 20:
                    lineas = pagina.extract_text_lines(layout=incluir_layout)
                    if lineas:
                        texto = "\n".join(line["text"] for line in lineas if line["text"].strip())
                
                # Opción 3: fallback a chars si todo lo anterior falla
                if not texto or len(texto.strip()) < 20:
                    chars = pagina.chars
                    if chars:
                        texto = "".join(c["text"] for c in chars)
                
                if texto and texto.strip():
                    texto_total.append(f"--- PÁGINA {i} ---\n{texto}\n")
                else:
                    texto_total.append(f"--- PÁGINA {i} --- (sin texto detectable)\n")
    
    except Exception as e:
        print(f"Error al abrir/procesar el PDF: {e}")
        return ""
    
    return "\n".join(texto_total)


# Uso
path = "../data/presupuesto_C.pdf"
texto_completo = extraer_texto_completo(path, incluir_layout=True)
print(texto_completo)

print("Primeros 5000 caracteres:\n")

# guardar todo en un archivo
with open("texto_extraido_pC.txt", "w", encoding="utf-8") as f:
    f.write(texto_completo)

print("\nTexto completo guardado en: texto_extraido_presupuesto.txt")

Total de páginas: 1
--- PÁGINA 1 ---
                                                                                    
           CARROCERIA Y PINTURA                                                     
                                        Documento no Valido como factura            
             "EQUIPO     C"                                                         
                                        PRESUPUESTO Nº 5028                         
          Echeandia 3084- C1406HRJ C.A.B.A    Fecha de emisión  1 de julio de 2025  
        Tel.4613-0720 - tallerequipoc@gmail.com                                     
               RESPONSABLE INSCRIPTO    C.U.I.T Nº 20-35316807-0 - Ing.Brutos Nº1619348-2 - Inic.Activ: 01/05/2022
  Señor:                                Cia. Seguro:                                
  Calle:                                Autom.Marca: CHEVROLET CRUZE                
  Localidad:                            Dominio:  AD 184 VR                        

In [7]:
# Diccionario de Sinónimos para normalizar columnas
SINONIMOS_COLUMNAS = {
    "part_number_oem": ["Código", "Cod.", "Nro Pieza", "Referencia"],
    "descripcion_taller": ["Descripción", "DESCRIPCION", "Concepto", "Detalle"],
    "precio": ["Precio ($)", "PRECIO", "Valor Unitario", "Precio Unit."],
    "cantidad": ["Cant.", "CANT.", "Cantidad", "Unidades"],
    "tipo": ["TIPO", "Tipo", "Categoría"]
}

def normalizar_cabeceras(columnas_pdf):
    mapeo_final = {}
    for col in columnas_pdf:
        col_limpia = str(col).strip()
        for estandar, lista_sinonimos in SINONIMOS_COLUMNAS.items():
            if col_limpia in lista_sinonimos:
                mapeo_final[col_limpia] = estandar
    return mapeo_final

In [42]:
# extraer patente/dominio con regex, considerando formato de Argentina
def extraer_patente_robusta(texto):
    patrones = [
        r'\b[A-Z]{2}\s?\d{3}\s?[A-Z]{2}\b', 
        r'\b[A-Z]{3}\s?\d{3}\b'
    ]
    for pat in patrones:
        match = re.search(pat, texto, re.IGNORECASE)
        if match:
            return match.group(0).upper().replace(" ", "")
    return "No detectada"


# extraer montos de mano de obra por "Dias de chapa" o "Paños de pintura"
# los talleres acostumbran colocar el total, no lo desglosan por pieza
def extraer_totales_mo(texto):
    
    def limpiar_a_numero(match):
        """Intenta convertir el match a número, si falla devuelve 0.0"""
        if not match:
            return 0.0
        
        try:
            valor_str = match.group(1)
            valor_num = valor_str.replace(" ", "").replace(",", ".")
            
            return float(valor_num)
        except (ValueError, IndexError, AttributeError):
            return 0.0

    regex_chapa = r"(?:CHAPA|Días Chapa).*?([\d\.]{3,10})"
    regex_pintura = r"(?:PINTURA|Paños Pintura).*?([\d\.]{3,10})"
    regex_trabajo = r"(?:MECANICA|MECANICA GENERAL 1|ELECTRICIDAD|TRABAJO DE TERCEROS|Mano de Obra)[:\s]*([\d\s\.,-]{3,10})"
    
    match_chapa = re.search(regex_chapa, texto, re.IGNORECASE)
    match_pintura = re.search(regex_pintura, texto, re.IGNORECASE)
    match_trabajo = re.search(regex_trabajo, texto, re.IGNORECASE)

    valor_chapa = limpiar_a_numero(match_chapa)
    valor_pintura = limpiar_a_numero(match_pintura)
    valor_trabajo = limpiar_a_numero(match_trabajo)

    return {
        "dias_chapa": valor_chapa,
        "pano_pintura": valor_pintura,
        "horas_trabajo": valor_trabajo
    }


# extraer el presupuesto detallado
def procesar_presupuesto_flexible(path_pdf):
    texto_completo = ""
    items_extraidos = []
    
    with pdfplumber.open(path_pdf) as pdf:
        for pagina in pdf.pages:
            texto_completo += (pagina.extract_text() or "") + "\n"
            tabla = pagina.extract_table()
            
            if not tabla or len(tabla) < 2: continue
            
            # Limpiar encabezados
            headers_raw = [str(c).replace('\n', ' ').strip() for c in tabla[0] if c is not None]
            mapeo = normalizar_cabeceras(headers_raw)
            
            if mapeo:
                # Obtenemos los índices reales que coinciden con nuestras columnas estándar
                indices_validos = {headers_raw.index(col_pdf): col_estandar 
                                  for col_pdf, col_estandar in mapeo.items()}
                
                for fila in tabla[1:]:
                    # --- FIX INDEX ERROR ---
                    # Verificamos que la fila tenga suficientes columnas
                    item = {}
                    for idx, col_estandar in indices_validos.items():
                        if idx < len(fila): # Solo si el índice existe en esta fila específica
                            item[col_estandar] = fila[idx]
                    
                    if item.get("descripcion_taller"):
                        items_extraidos.append(item)

    totales_mo = extraer_totales_mo(texto_completo)
    
    return {
        "metadata": {
            "patente": extraer_patente_robusta(texto_completo),
            "totales_extra": totales_mo
        },
        "items": items_extraidos
    }


try:
    path = "../data/presupuesto_A2.pdf" 
    resultado = procesar_presupuesto_flexible(path)
    print(f"Patente: {resultado['metadata']['patente']}")
    print(f"Totales MO: {resultado['metadata']['totales_extra']}")
    print(f"Items encontrados: {len(resultado['items'])}")
except Exception as e:
    print(f"Error al procesar: {e}")

df_items = pd.DataFrame(resultado['items'])
print(f"\nItems encontrados:{len(df_items)}")
print(df_items.to_string(index=False))

Patente: AH486PS
Totales MO: {'dias_chapa': 6.0, 'pano_pintura': 7.5, 'horas_trabajo': 1.0}
Items encontrados: 18

Items encontrados:18
    tipo        descripcion_taller cantidad     precio
Servicio         CHAPA CARGO C.SEG     6.00  154545.45
Servicio       PINTURA CARGO C.SEG     7.50  158677.69
Servicio       TRABAJO DE TERCEROS     1.00  165289.26
Repuesto PANEL SUB-ASSY, BACK DOOR     1.00 1904631.53
Repuesto                   EMBLEMA     1.00   72480.69
Repuesto                   EMBLEMA     1.00   38907.38
Repuesto                   EMBLEMA     1.00   70127.42
Repuesto           EMBLEMA COROLLA     1.00   91855.94
Repuesto          COVER, RR BUMPER     1.00  549357.63
Repuesto RETENEDOR LAT.PARAG.RR LH     1.00  133456.51
Repuesto RETENEDOR LAT.PARAG.RR RH     1.00  133456.51
Repuesto     EXTENSION BOMPER TRAS     1.00  210970.59
Repuesto     EXTENSION BOMPER TRAS     1.00  311206.78
Repuesto      CUBIERTA BOMPER TRAS     1.00   13086.79
Repuesto      CUBIERTA BUMPER TRAS     

```json
{
  "metadata": {
o    "place": "Nombre Taller",
x    "patente": "AB123CD",
o    "objetivo": "inspeccion",
o    "fecha_extraccion": "2024-05-20"
  },
  "items_piezas": [
    {
      "id_utrace": 160,
      "descripcion_oficial": "Óptica Delantera Derecha",
      "descripcion_taller": "UNIT HEADLAMP W GAS",
      "tipo": "Pieza",
      "sustitucion": true,
      "cantidad": 1.0,
x      "dias_chapa": 0.0,
x      "pano_pintura": 0.0,
o      "horas_trabajo": 0.0,
      "costo_ars": 0.0,
o      "observaciones_inspec": "Golpe profundo en en lateral derecho.",
      "confianza_match": 85
    }
  ]
}
```